# Fine tuning version

based on: https://github.com/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb (IMDB sentiment analysis) | https://towardsdatascience.com/the-guide-to-multi-tasking-with-the-t5-transformer-90c70a08837b (The Guide to Multi-Tasking with the T5 Transformer) | https://discuss.huggingface.co/t/t5-classification-using-text2text/504/8 (SST2 sentiment analysis classification)

# Imports

In [1]:
import sys
!{sys.executable} -m pip install --user pytorch_lightning

The directory '/home/atroska/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/atroska/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 19.0.3, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /home/atroska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


# Methods

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(seed)

In [4]:
def load_tweet(path: str, label: int):
    data = []
    with open(path) as file:
        for line in file:
            data.append(line)
    data_df = pd.DataFrame(data, columns = {'tweet'})
    data_df['label'] = label
    print('Loaded data: ' + path)
    return data_df

In [5]:
def concat_data(dset1, dset2):
    df = pd.concat([dset1, dset2])
    print('Concatenated data')
    return df

In [6]:
def preprocess_tweet(tweet):
    '''
    :param tweet: tweet as a string
    :return: string s which is preprocessed tweet
    '''
    tweet = re.sub('<user>', '', tweet) # remove user tags
    tweet = re.sub('<url>', '', tweet) # remove url tags
    tweet = re.sub('#\w*', '', tweet) # remove hashtags
    tweet = re.sub('[0-9]', '', tweet) # remove numbers
    tweet = re.sub('[^\w\s]', '', tweet) # remove punctuation
    tweet = re.sub('\s+', ' ', tweet) # remove excess whitespace
    tweet = re.sub('^\s', '', tweet) # remove excess whitespace
    tweet = re.sub('\s$', '', tweet) # remove excess whitespace
    tweet += '\n'
    #tweet = ' '.join(ws.segment(tweet)) # segment words
    #tweet = ' '.join([speller(w) for w in tweet.split()]) # spell checking
    return tweet

In [7]:
def sst2_text_processing(tweet):
    #add the correct prefix in order to use pretrained t5 model
    new_tweet = "sst2 sentence: " + tweet
    return new_tweet
    

In [8]:
def accuracy(true, preds):
    from sklearn.metrics import accuracy_score
    if len(preds) != len(true):
        print("ERROR: preds and true don't have the same length")
        return 0
    else:
        return accuracy_score(true, preds, normalize=False)

# Main

In [9]:
#POSITIVE_PATH = '/cluster/home/alefevre/data/twitter/Datas/train_pos.txt'
#NEGATIVE_PATH  = '/cluster/home/alefevre/data/twitter/Datas/train_neg.txt'
#OUTPUT_PREDS_PATH = '/cluster/home/alefevre/results/twitter/XLNET_test_preds.txt'
#PRETRAINED_PATH = 'xlnet-base-cased'

POSITIVE_PATH = '/home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_pos.txt'
NEGATIVE_PATH  = '/home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_neg.txt'
OUTPUT_PREDS_PATH = '/home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/T5google_test_preds.txt'
N_EPOCHS = 5 # or 3 ?
BATCH_SIZE = 16
TEST_SIZE = 0.10


set_seed(42)


#load datas
pos_df = load_tweet(POSITIVE_PATH, 1)
neg_df = load_tweet(NEGATIVE_PATH, 0)

#concat datas
all_tweet_df = concat_data(pos_df, neg_df)

#apply basic preprocessing (tobias)
all_tweet_df['tweet'] = all_tweet_df['tweet'].apply(lambda row: preprocess_tweet(str(row)))

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Loaded data: /home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_pos.txt
Loaded data: /home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_neg.txt
Concatenated data


In [10]:
all_tweet_df

,tweet,label
0,i dunno justin read my mention or not only jus...,1
1,because your logic is so dumb i wont even crop...,1
2,just put casper in a box looved the battle\n,1
3,thanks sir dont trip lil mama just keep doin y...,1
4,visiting my brother tmr is the bestest birthda...,1
...,...,...
99995,cant wait to fake tan tonight hate being pale\n,0
99996,darling i lost my internet connection and its ...,0
99997,kanguru defender basic gb usb flash drive kdfb...,0
99998,rizan is sad now\n,0


# SST-2 Binary Classification

In [11]:
import torch
import torch.nn.functional as F
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [12]:
tokenizer = T5Tokenizer.from_pretrained("t5-11b")
model = T5ForConditionalGeneration.from_pretrained("t5-11b")
model.eval()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/spiece.model HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140615543461928 on /home/atroska/.cache/huggingface/transformers/0172c8f05db06fdc1d9f5be691fa907b7da289cf4b777506b956dc76d9bf1ceb.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
INFO:filelock:Lock 140615543461928 acquired on /home/atroska/.cache/huggingface/transformers/0172c8f05db06fdc1d9f5be691fa907b7da289cf4b777506b956dc76d9bf1ceb.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /t5-11b/resolve/main/spiece.model HTTP/1.1" 200 791656


DEBUG:filelock:Attempting to release lock 140615543461928 on /home/atroska/.cache/huggingface/transformers/0172c8f05db06fdc1d9f5be691fa907b7da289cf4b777506b956dc76d9bf1ceb.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
INFO:filelock:Lock 140615543461928 released on /home/atroska/.cache/huggingface/transformers/0172c8f05db06fdc1d9f5be691fa907b7da289cf4b777506b956dc76d9bf1ceb.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/special_tokens_map.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/tokenizer.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140615419279736 on /home/atroska/.cache/huggingface/transformers/198eb2e52771632f85a91599d03d2c3c7a2d13c46f168d2a280284affb64d4e6.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
INFO:filelock:Lock 140615419279736 acquired on

DEBUG:filelock:Attempting to release lock 140615419279736 on /home/atroska/.cache/huggingface/transformers/198eb2e52771632f85a91599d03d2c3c7a2d13c46f168d2a280284affb64d4e6.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
INFO:filelock:Lock 140615419279736 released on /home/atroska/.cache/huggingface/transformers/198eb2e52771632f85a91599d03d2c3c7a2d13c46f168d2a280284affb64d4e6.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140610960289520 on /home/atroska/.cache/huggingface/transformers/81569faf106ccedd12b4204414bfaf719ce664c51e4192954c3e2a45c2b6183d.f3d4f63c19654eaafefa7926880e38ff43a93df01b4df7b7b60c00bb1b10e9a6.lock
INFO:filelock:Lock 140610960289520 acquired on /home/atroska/.cache/huggingface/transformers/81569faf106ccedd12b4204414bfaf719ce664c51e4192954c3e2a45c2b6183d.f3d4f63c19654eaafefa7926880e38ff43a93df01b4df7b7b60c00bb1b10e9a6.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /t5-11b/resolve/main/config.json HTTP/1.1" 200 1203


DEBUG:filelock:Attempting to release lock 140610960289520 on /home/atroska/.cache/huggingface/transformers/81569faf106ccedd12b4204414bfaf719ce664c51e4192954c3e2a45c2b6183d.f3d4f63c19654eaafefa7926880e38ff43a93df01b4df7b7b60c00bb1b10e9a6.lock
INFO:filelock:Lock 140610960289520 released on /home/atroska/.cache/huggingface/transformers/81569faf106ccedd12b4204414bfaf719ce664c51e4192954c3e2a45c2b6183d.f3d4f63c19654eaafefa7926880e38ff43a93df01b4df7b7b60c00bb1b10e9a6.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /t5-11b/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 140615419279064 on /home/atroska/.cache/huggingface/transformers/3ec200f21984c6b177d08534a7166201616fe542ddd5c6e61927f7908bf9f75f.200226855d13a9f5ec82e28b352f6f771748fba71f240e65cd3dfe99889b4ccc.lock
INFO:filelock:Lock 140615419279064 acquired on /home/atroska/.cache/huggingface/transformers/3ec200f21984c6b177d08534a7166201616fe542ddd5c6e61927f7908bf9f75f.200226855d13a9f5ec82e28b352f6f771748fba71f240e65cd3dfe99889b4ccc.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "GET /lfs.huggingface.co/t5-11b/5fdc64177b14b0f72437fea171a752c626733f67755842528c75b90cacd1c807 HTTP/1.1" 200 45229452544


DEBUG:filelock:Attempting to release lock 140615419279064 on /home/atroska/.cache/huggingface/transformers/3ec200f21984c6b177d08534a7166201616fe542ddd5c6e61927f7908bf9f75f.200226855d13a9f5ec82e28b352f6f771748fba71f240e65cd3dfe99889b4ccc.lock
INFO:filelock:Lock 140615419279064 released on /home/atroska/.cache/huggingface/transformers/3ec200f21984c6b177d08534a7166201616fe542ddd5c6e61927f7908bf9f75f.200226855d13a9f5ec82e28b352f6f771748fba71f240e65cd3dfe99889b4ccc.lock


KeyboardInterrupt: 

## Example

In [17]:
text = "sst2 sentence: it confirms fincher ’s status as a film maker who artfully bends technical know-how to the service of psychological insight"

In [19]:
with torch.no_grad():
    enc = tokenizer(text, return_tensors="pt")
    decoder_input_ids = torch.tensor([tokenizer.pad_token_id]).unsqueeze(0) 
    logits = model(**enc, decoder_input_ids=decoder_input_ids)[0]
    tokens = torch.argmax(logits, dim=2)
    sentiments = tokenizer.batch_decode(tokens)
    # 'positve'

In [20]:
sentiments

['positive']

## On our dataset

In [19]:
#load datas
pos_df = load_tweet(POSITIVE_PATH, 1)
neg_df = load_tweet(NEGATIVE_PATH, 0)

#concat datas
all_tweet_df = concat_data(pos_df, neg_df)

#apply basic preprocessing (tobias)
all_tweet_df['tweet'] = all_tweet_df['tweet'].apply(lambda row: preprocess_tweet(str(row)))
all_tweet_df['tweet'] = all_tweet_df['tweet'].apply(lambda row: sst2_text_processing(str(row)))


Loaded data: /home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_pos.txt
Loaded data: /home/atroska/Documents/Zurich/MA2/CIL/Project/Datas/twitter-datasets/train_neg.txt
Concatenated data


In [20]:
all_tweet_df

,tweet,label
0,sst2 sentence: i dunno justin read my mention ...,1
1,sst2 sentence: because your logic is so dumb i...,1
2,sst2 sentence: just put casper in a box looved...,1
3,sst2 sentence: thanks sir dont trip lil mama j...,1
4,sst2 sentence: visiting my brother tmr is the ...,1
...,...,...
99995,sst2 sentence: cant wait to fake tan tonight h...,0
99996,sst2 sentence: darling i lost my internet conn...,0
99997,sst2 sentence: kanguru defender basic gb usb f...,0
99998,sst2 sentence: rizan is sad now\n,0


In [21]:
all_tweet_df_randomize =  all_tweet_df.sample(frac=1)

In [22]:
all_tweet_df_randomize

,tweet,label
96900,sst2 sentence: fallout audio cd front line ass...,0
43323,sst2 sentence: because i have a legit job\n,1
71194,sst2 sentence: genetics a conceptual approach ...,0
85518,sst2 sentence: yeah half the prob is my class ...,0
10289,sst2 sentence: do one megazza just because nob...,1
...,...,...
84703,sst2 sentence: thomas guide northern virginia ...,0
97619,sst2 sentence: and more recently too\n,1
28597,sst2 sentence: rt according to forbes here are...,0
71897,sst2 sentence: orange can make u sneeze more r...,1


In [23]:
T5_preds_randomize = []

In [ ]:
counter = 0
for i in all_tweet_df_randomize['tweet']:
    counter += 1
    if(counter%100==0):
        print("Iteration: " + str(counter))
    with torch.no_grad():
        enc = tokenizer(i, return_tensors="pt")
        decoder_input_ids = torch.tensor([tokenizer.pad_token_id]).unsqueeze(0) 
        logits = model(**enc, decoder_input_ids=decoder_input_ids)[0]
        tokens = torch.argmax(logits, dim=2)
        sentiments = tokenizer.batch_decode(tokens)
    T5_preds_randomize.append(sentiments)

Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900
Iteration: 1000
Iteration: 1100
Iteration: 1200
Iteration: 1300
Iteration: 1400
Iteration: 1500
Iteration: 1600
Iteration: 1700
Iteration: 1800
Iteration: 1900
Iteration: 2000
Iteration: 2100
Iteration: 2200
Iteration: 2300
Iteration: 2400
Iteration: 2500
Iteration: 2600
Iteration: 2700
Iteration: 2800
Iteration: 2900
Iteration: 3000
Iteration: 3100
Iteration: 3200
Iteration: 3300
Iteration: 3400
Iteration: 3500
Iteration: 3600
Iteration: 3700
Iteration: 3800
Iteration: 3900
Iteration: 4000
Iteration: 4100
Iteration: 4200
Iteration: 4300
Iteration: 4400
Iteration: 4500
Iteration: 4600
Iteration: 4700
Iteration: 4800
Iteration: 4900
Iteration: 5000
Iteration: 5100
Iteration: 5200
Iteration: 5300
Iteration: 5400
Iteration: 5500
Iteration: 5600
Iteration: 5700
Iteration: 5800
Iteration: 5900
Iteration: 6000
Iteration: 6100
Iteration: 6200
Iteration: 6300
I

Iteration: 49000
Iteration: 49100
Iteration: 49200
Iteration: 49300
Iteration: 49400
Iteration: 49500
Iteration: 49600
Iteration: 49700
Iteration: 49800
Iteration: 49900
Iteration: 50000
Iteration: 50100
Iteration: 50200
Iteration: 50300
Iteration: 50400
Iteration: 50500
Iteration: 50600
Iteration: 50700
Iteration: 50800
Iteration: 50900
Iteration: 51000
Iteration: 51100
Iteration: 51200
Iteration: 51300
Iteration: 51400
Iteration: 51500
Iteration: 51600
Iteration: 51700
Iteration: 51800
Iteration: 51900
Iteration: 52000
Iteration: 52100
Iteration: 52200
Iteration: 52300
Iteration: 52400
Iteration: 52500
Iteration: 52600
Iteration: 52700
Iteration: 52800
Iteration: 52900
Iteration: 53000
Iteration: 53100
Iteration: 53200
Iteration: 53300
Iteration: 53400
Iteration: 53500
Iteration: 53600
Iteration: 53700
Iteration: 53800
Iteration: 53900
Iteration: 54000
Iteration: 54100
Iteration: 54200
Iteration: 54300
Iteration: 54400
Iteration: 54500
Iteration: 54600
Iteration: 54700
Iteration: 548

In [36]:
true = all_tweet_df_randomize['label']

In [47]:
true_list = list(true)

In [48]:
true_list

[0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,


In [38]:
true_np = np.array(true)

In [39]:
true_np

array([0, 1, 1, ..., 0, 0, 0])

In [40]:
T5_preds_randomize

[['negative'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['negative'],
 ['positive'],
 ['positive'],
 ['negative'],
 ['negative'],
 ['negativ

In [45]:
T5_preds_randomize_labels = []
for i in T5_preds_randomize:
    if i[0] == "negative":
        T5_preds_randomize_labels.append(0)
    else:
        T5_preds_randomize_labels.append(1)

In [46]:
T5_preds_randomize_labels

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,


In [50]:
true_list_samelen = true_list[0:len(T5_preds_randomize_labels)]

In [55]:
acc = accuracy(true_list_samelen, T5_preds_randomize_labels)

In [57]:
acc/len(true_list_samelen)*100

64.85671492760858